### Import Libraries

In [1]:
import numpy as np 
import pandas as pd 
import boto3

from sqlalchemy import create_engine
from sqlalchemy import text
import psycopg2

### User & Data AWS Redshift Credentials

In [2]:
redshift_username = "rutvik_bhende"
redshift_password = "B08tLOo-n69d5t4"
port = 5439

## ISC creds
redshift_endpoint_isc = "isc-prd-data-warehouse.c3aww65gl0dd.us-east-1.redshift.amazonaws.com"
dbname_isc = "isc"
## HC creds
redshift_endpoint_hc = "data-warehouse.aws.healthcare.com"
dbname_hc = "hc"
## PH creds
redshift_endpoint_ph = "data-warehouse.pivothealth.com"
dbname_ph = "ph"



# Load all relevant datasets

### MA Post-Conv LTV Model Predictions data

In [3]:
post_conv_df = pd.read_csv("data/MA LTV model predictionsFINAL.csv", low_memory=False)
post_conv_df

,application_id,medicare_number,cancellation_model_prediction,probability_of_cancellation,duration_model_prediction,LTV,sk_effective_date,sk_end_date,sk_submitted_date,sk_termination_date,...,2022_star_ratings_overall,screenings_tests_vaccines,managing_chronic_conditions,experience_with_plan,member_complaints,customer_service,number_of_plans_available,carrier_competition_orig,carrier_competition,sk_owner_gender
0,0064V000013sHbAQAU,7g20tn9jv07,1,0.821085,0,$0,2021-02-01 0:00,NaN,2021-01-26 0:00,2021-02-01 0:00,...,4.5,4,4,4,5,5,240,High,3,10000
1,e8502590-3832-46f9-97eb-7a7627d6eed9,2P75RK2UN65,0,0.000570,44,"$1,682",2022-01-01 0:00,NaN,2021-12-06 0:00,NaN,...,4.0,3,3,4,4,5,94,Low,1,10008
2,6aaf4239-02ec-47b8-88e7-003e5d81f479,1nq6kh8aj68,0,0.000687,36,"$1,439",2022-02-01 0:00,NaN,2022-01-11 0:00,NaN,...,4.0,3,3,4,4,5,337,High,3,10008
3,0064V000013rtmBQAQ,9H05-F09-MY70,0,0.162569,33,"$1,348",2020-10-01 0:00,NaN,2020-09-24 0:00,2020-10-01 0:00,...,4.0,3,3,4,4,5,126,Medium,2,10008
4,cbcbc7d7-fb7c-4865-91fa-4c8da62fb8ef,6ad3pu7mw80,0,0.000383,38,"$1,508",2022-01-01 0:00,NaN,2021-11-12 0:00,NaN,...,4.0,3,4,4,4,5,583,High,3,10004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15604,1210ffae-a29e-42b9-998b-3161f9bc524e,2NA5TG2WW85,0,0.000397,41,"$1,597",2022-02-01 0:00,NaN,2022-01-24 0:00,NaN,...,4.0,3,4,4,4,5,337,High,3,10004
15605,abb65a39-a65d-4d37-8046-94b8a5db3254,3GJ5R95HC67,0,0.000480,42,"$1,627",2022-02-01 0:00,NaN,2022-01-27 0:00,NaN,...,4.0,3,4,4,4,5,126,Medium,2,10004
15606,b881bd5f-5f03-44d9-bebc-6cde0dc4624a,4FK2E02XN07,0,0.000376,41,"$1,597",2022-02-01 0:00,NaN,2022-01-11 0:00,NaN,...,4.0,3,4,4,4,5,337,High,3,10008
15607,04e4273b-5202-4a36-b365-64698ff06356,9CE4NQ9TH88,0,0.000435,40,"$1,567",2022-02-01 0:00,NaN,2022-01-24 0:00,NaN,...,4.0,3,4,4,4,5,202,High,3,10004


### Create AWS Redshift engines

In [6]:
## ISC
isc_engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
                % (redshift_username, redshift_password, redshift_endpoint_isc, port, dbname_isc)
isc_engine = create_engine(isc_engine_string)

## HC
hc_engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
                % (redshift_username, redshift_password, redshift_endpoint_hc, port, dbname_hc)
hc_engine = create_engine(hc_engine_string)

### SQL String: Total MEDICARE ADVANTAGE Applications from ISC Data Cluster

In [10]:
MA_appls_all_sql = """
SELECT
    fapp.application_id,
    fapp.owner_email,
    fapp.application_name,
    fapp.policy_id,
    fapp.owner_id,
    fapp.owner_phone,
    fapp.sk_submitted_date,
    fapp.sk_owner_gender,
    fapp.owner_name,
    fapp.sk_date_of_birth,
    fapp.zip_code AS app_zip_code,
    fpol.zip_code AS pol_zip_code,
    fapp.parent_application_id,
    fapp.sk_referral_flag,
    dprod.bk_product_type,
    dcarr.carrier,
    dgen.bk_gender,
    fc.lead_id
FROM isc_dm_sales.f_application fapp
    LEFT JOIN isc_dm_sales.f_policy fpol
                    ON fapp.policy_id=fpol.policy_id
    INNER JOIN isc_dm_sales.d_product dprod
                    ON fapp.sk_product = dprod.sk_product
    INNER JOIN isc_dm_sales.d_carrier dcarr
                    ON fapp.sk_carrier = dcarr.sk_carrier
    INNER JOIN isc_dm_sales.d_gender dgen
                    ON fapp.sk_owner_gender = dgen.sk_gender
    LEFT JOIN isc_dm_sales.d_status ds
                    ON ds.sk_status = fpol.sk_policy_status
    LEFT JOIN isc_dm_sales.f_calls fc
                    ON fc.application_id = fapp.parent_application_id
WHERE bk_product IN (1048,1019,1020, 1052);
"""

### Fetch MEDICARE ADVANTAGE Applications data

In [11]:
MA_appls_all = pd.read_sql_query(text(MA_appls_all_sql), isc_engine)
MA_appls_all

,application_id,owner_email,application_name,policy_id,owner_id,owner_phone,sk_submitted_date,sk_owner_gender,owner_name,sk_date_of_birth,app_zip_code,pol_zip_code,parent_application_id,sk_referral_flag,bk_product_type,carrier,bk_gender,lead_id
0,e393aa2c-cb6e-480b-9caf-502ab2165b6c,rodwilli@outlook.com,Humana Gold Plus H4141-015 (HMO),None,6yj3x36hd88,2068542073,20220308,10008,Rodney Williams,19520420.0,30253,None,e393aa2c-cb6e-480b-9caf-502ab2165b6c,0,O65,O65 - Humana,M,123044397.0
1,e393aa2c-cb6e-480b-9caf-502ab2165b6c,rodwilli@outlook.com,Humana Gold Plus H4141-015 (HMO),None,6yj3x36hd88,2068542073,20220308,10008,Rodney Williams,19520420.0,30253,None,e393aa2c-cb6e-480b-9caf-502ab2165b6c,0,O65,O65 - Humana,M,123044397.0
2,e393aa2c-cb6e-480b-9caf-502ab2165b6c,rodwilli@outlook.com,Humana Gold Plus H4141-015 (HMO),None,6yj3x36hd88,2068542073,20220308,10008,Rodney Williams,19520420.0,30253,None,e393aa2c-cb6e-480b-9caf-502ab2165b6c,0,O65,O65 - Humana,M,123044397.0
3,e393aa2c-cb6e-480b-9caf-502ab2165b6c,rodwilli@outlook.com,Humana Gold Plus H4141-015 (HMO),None,6yj3x36hd88,2068542073,20220308,10008,Rodney Williams,19520420.0,30253,None,e393aa2c-cb6e-480b-9caf-502ab2165b6c,0,O65,O65 - Humana,M,123044397.0
4,0064V000013qyk2QAA,owyndietert101@gmail.com,HMO,None,0014V00001mnX63QAE,2102028271,20201231,10008,Owyn Dietert,19860904.0,78205,None,0064V000013qyk2QAA,0,O65,O65 - Aetna,M,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72991,67e5327a-5bad-4af7-bc40-8cd5a50f36f6,None,Humana Gold Plus H6622-054 (HMO),108094233,2wx4ye5mp88,7249307234,20211201,10004,Jacqueline Morgan,19310505.0,15012,15012,4dfb4a01-c8b5-4555-b9dc-39e3974a3ad9,0,O65,O65 - Humana,F,NaN
72992,8b595599-4191-427c-ad1c-77e8fcd5e637,amohtom@yahoo.com,HumanaChoice H5525-051 (PPO),107993346,7dr9yw7vn42,2158408295,20211126,10008,Tom Amoh,19450114.0,19095,19095,8b595599-4191-427c-ad1c-77e8fcd5e637,0,O65,O65 - Humana,M,NaN
72993,feac6fd7-9d73-4348-af99-736d2b69bb4d,None,HumanaChoice H5216-254 (PPO),108650885,7w89c55eg42,4027277048,20220209,10004,Virginia Brushwood,19320813.0,68025,68025,feac6fd7-9d73-4348-af99-736d2b69bb4d,0,O65,O65 - Humana,F,NaN
72994,347a0253-74db-444b-b483-9d68e6235c95,None,Anthem MediBlue Access Basic (PPO),4QV3JJ1UD07,4qv3jj1ud07,4788754331,20211015,10008,Alphonso Crafton,19541022.0,31019,31019,0064V00001HPm4mQAD,0,O65,O65 - Anthem,M,NaN


In [14]:
## Drop all Duplicated row entries
MA_appls_df = MA_appls_all.drop_duplicates(keep='last', ignore_index=True)
MA_appls_df

,application_id,owner_email,application_name,policy_id,owner_id,owner_phone,sk_submitted_date,sk_owner_gender,owner_name,sk_date_of_birth,app_zip_code,pol_zip_code,parent_application_id,sk_referral_flag,bk_product_type,carrier,bk_gender,lead_id
0,c3776ff0-a7b6-4fc0-a438-82ca18834884,None,Aetna Medicare Dual Complete Plan (HMO D-SNP),None,4nu8c34hn34,2142823046,20211121,10008,John Lindsey,19431209.0,75173,None,c3776ff0-a7b6-4fc0-a438-82ca18834884,0,O65,O65 - Aetna,M,NaN
1,0064V000013sGFZQA2,None,PPO,None,0014V00001px2k8QAA,2489347965,20210223,10008,John Ebarra,19350617.0,48083,None,0064V000013sGFZQA2,0,O65,O65 - Priority Health,M,NaN
2,0064V00001BezKUQAZ,None,PPO,None,0014V00001vW9u6QAC,2525862957,20210330,10008,Bernanrd Held,19420121.0,27850,None,0064V00001BezKUQAZ,0,O65,O65 - Wellcare,M,NaN
3,fd74b67c-d8da-4248-95e3-878378f55741,battles_c@bellsouth.net,Cigna True Choice Medicare (PPO),None,3un6t83dy58,2563903277,20211025,10008,A.B Battles,19410210.0,35907,None,fd74b67c-d8da-4248-95e3-878378f55741,0,O65,O65 - Cigna,M,NaN
4,035f80cf-4b17-4a4b-ae0c-845351fb3f6a,None,Aetna Medicare Assure Premier (HMO D-SNP),None,7w71ve9fj23,3135448170,20211101,10008,Lester Williams,19550323.0,48207,None,035f80cf-4b17-4a4b-ae0c-845351fb3f6a,0,O65,O65 - Aetna,M,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20980,67e5327a-5bad-4af7-bc40-8cd5a50f36f6,None,Humana Gold Plus H6622-054 (HMO),108094233,2wx4ye5mp88,7249307234,20211201,10004,Jacqueline Morgan,19310505.0,15012,15012,4dfb4a01-c8b5-4555-b9dc-39e3974a3ad9,0,O65,O65 - Humana,F,NaN
20981,8b595599-4191-427c-ad1c-77e8fcd5e637,amohtom@yahoo.com,HumanaChoice H5525-051 (PPO),107993346,7dr9yw7vn42,2158408295,20211126,10008,Tom Amoh,19450114.0,19095,19095,8b595599-4191-427c-ad1c-77e8fcd5e637,0,O65,O65 - Humana,M,NaN
20982,feac6fd7-9d73-4348-af99-736d2b69bb4d,None,HumanaChoice H5216-254 (PPO),108650885,7w89c55eg42,4027277048,20220209,10004,Virginia Brushwood,19320813.0,68025,68025,feac6fd7-9d73-4348-af99-736d2b69bb4d,0,O65,O65 - Humana,F,NaN
20983,347a0253-74db-444b-b483-9d68e6235c95,None,Anthem MediBlue Access Basic (PPO),4QV3JJ1UD07,4qv3jj1ud07,4788754331,20211015,10008,Alphonso Crafton,19541022.0,31019,31019,0064V00001HPm4mQAD,0,O65,O65 - Anthem,M,NaN


In [17]:
MA_appls_df.lead_id.notna().sum()

4646

### Merge MEDICARE ADVANTAGE data with Post-Conv LTV preds data

In [ ]:
MA_PostConv_df = pd.merge(
    MA_appls_df, 
    post_conv_df, 
    on= ['application_id', 'policy_id'],
    how= 'left',
    copy=True
    )

In [42]:
print(pd.merge(
    MA_appls_df, 
    post_conv_df, 
    on= ['application_id', 'policy_id'],
    how= 'left',
    copy=True
    ).iloc[0])

application_id                        c3776ff0-a7b6-4fc0-a438-82ca18834884
owner_email_x                                                         None
application_name_x           Aetna Medicare Dual Complete Plan (HMO D-SNP)
policy_id                                                             None
owner_id_x                                                     4nu8c34hn34
                                                 ...                      
customer_service                                                       NaN
number_of_plans_available                                              NaN
carrier_competition_orig                                               NaN
carrier_competition                                                    NaN
sk_owner_gender_y                                                      NaN
Name: 0, Length: 162, dtype: object


In [39]:
post_conv_df

,application_id,medicare_number,cancellation_model_prediction,probability_of_cancellation,duration_model_prediction,LTV,sk_effective_date,sk_end_date,sk_submitted_date,sk_termination_date,...,2022_star_ratings_overall,screenings_tests_vaccines,managing_chronic_conditions,experience_with_plan,member_complaints,customer_service,number_of_plans_available,carrier_competition_orig,carrier_competition,sk_owner_gender
0,0064V000013sHbAQAU,7g20tn9jv07,1,0.821085,0,$0,2021-02-01 0:00,NaN,2021-01-26 0:00,2021-02-01 0:00,...,4.5,4,4,4,5,5,240,High,3,10000
1,e8502590-3832-46f9-97eb-7a7627d6eed9,2P75RK2UN65,0,0.000570,44,"$1,682",2022-01-01 0:00,NaN,2021-12-06 0:00,NaN,...,4.0,3,3,4,4,5,94,Low,1,10008
2,6aaf4239-02ec-47b8-88e7-003e5d81f479,1nq6kh8aj68,0,0.000687,36,"$1,439",2022-02-01 0:00,NaN,2022-01-11 0:00,NaN,...,4.0,3,3,4,4,5,337,High,3,10008
3,0064V000013rtmBQAQ,9H05-F09-MY70,0,0.162569,33,"$1,348",2020-10-01 0:00,NaN,2020-09-24 0:00,2020-10-01 0:00,...,4.0,3,3,4,4,5,126,Medium,2,10008
4,cbcbc7d7-fb7c-4865-91fa-4c8da62fb8ef,6ad3pu7mw80,0,0.000383,38,"$1,508",2022-01-01 0:00,NaN,2021-11-12 0:00,NaN,...,4.0,3,4,4,4,5,583,High,3,10004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15604,1210ffae-a29e-42b9-998b-3161f9bc524e,2NA5TG2WW85,0,0.000397,41,"$1,597",2022-02-01 0:00,NaN,2022-01-24 0:00,NaN,...,4.0,3,4,4,4,5,337,High,3,10004
15605,abb65a39-a65d-4d37-8046-94b8a5db3254,3GJ5R95HC67,0,0.000480,42,"$1,627",2022-02-01 0:00,NaN,2022-01-27 0:00,NaN,...,4.0,3,4,4,4,5,126,Medium,2,10004
15606,b881bd5f-5f03-44d9-bebc-6cde0dc4624a,4FK2E02XN07,0,0.000376,41,"$1,597",2022-02-01 0:00,NaN,2022-01-11 0:00,NaN,...,4.0,3,4,4,4,5,337,High,3,10008
15607,04e4273b-5202-4a36-b365-64698ff06356,9CE4NQ9TH88,0,0.000435,40,"$1,567",2022-02-01 0:00,NaN,2022-01-24 0:00,NaN,...,4.0,3,4,4,4,5,202,High,3,10004


In [35]:
c= list()
t = [col for col in post_conv_df.columns if col in MA_appls_df.columns]
# MA_appls_df.columns

In [36]:
t

['application_id',
 'sk_submitted_date',
 'sk_date_of_birth',
 'owner_email',
 'owner_phone',
 'owner_id',
 'policy_id',
 'application_name',
 'bk_product_type',
 'carrier',
 'bk_gender',
 'sk_owner_gender']

### ZipCode Enriched Data

In [14]:
zip_enriched_sql = """
SELECT * FROM data_science.zipcode_data_enriched;
"""
zip_enriched = pd.read_sql_query(text(zip_enriched_sql), hc_engine)
zip_enriched

,zipcode,state,lat,lng,timezone,radius_in_miles,population,population_density,land_area_in_sqmi,water_area_in_sqmi,...,physical_health,physical_inactivity,sleep_lt_7_hours,stroke,taking_bp_medication,teeth_loss,ma_penetration_2020,ma_penetration_2021,delta_ma_enrolled,delta_ma_penetration_points
0,36567,AL,30.5616,-87.6373,Central,12.0,12672.0,55.0,228.46,1.45,...,12.9,24.90,35.10,3.8,79.90,13.10,48.61,51.60,2584.0,2.99
1,36574,AL,30.5048,-87.4659,Central,4.0,1341.0,96.0,14.01,0.76,...,12.9,24.90,35.10,3.8,79.90,13.10,48.61,51.60,2584.0,2.99
2,36576,AL,30.5222,-87.7457,Central,3.0,4143.0,213.0,19.45,0.06,...,12.9,24.90,35.10,3.8,79.90,13.10,48.61,51.60,2584.0,2.99
3,36577,AL,30.6749,-87.9153,America/Anchorage,NaN,17603.0,2536.0,NaN,NaN,...,13.3,26.80,39.90,3.3,66.20,22.00,1.33,1.72,178.0,0.39
4,36578,AL,30.7476,-87.7787,Central,5.0,1749.0,57.0,30.61,0.22,...,12.9,24.90,35.10,3.8,79.90,13.10,48.61,51.60,2584.0,2.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40967,26452,WV,39.0411,-80.4737,Eastern,13.0,9676.0,78.0,124.69,0.87,...,19.0,31.80,42.20,4.5,80.40,23.90,41.03,45.67,194.0,4.64
40968,25501,WV,38.1609,-81.9840,Eastern,6.0,1130.0,45.0,25.00,0.01,...,21.7,35.00,44.40,5.0,81.00,27.60,50.07,55.08,274.0,5.01
40969,25506,WV,38.2694,-82.2684,Eastern,11.0,5037.0,48.0,104.08,0.52,...,19.2,30.75,42.85,4.5,79.55,23.75,50.07,55.08,274.0,5.01
40970,25521,WV,38.2197,-81.9995,Eastern,4.0,1101.0,36.0,30.91,0.01,...,21.7,35.00,44.40,5.0,81.00,27.60,50.07,55.08,274.0,5.01


In [16]:
zip_enriched.columns

Index(['zipcode', 'state', 'lat', 'lng', 'timezone', 'radius_in_miles',
       'population', 'population_density', 'land_area_in_sqmi',
       'water_area_in_sqmi', 'housing_units', 'occupied_housing_units',
       'median_home_value', 'median_household_income', 'annual_checkup',
       'arthritis', 'binge_drinking', 'copd', 'cancer_except_skin',
       'cervical_cancer_screening', 'cholesterol_screening',
       'chronic_kidney_disease', 'colorectal_cancer_screening',
       'core_preventive_services_for_older_men',
       'core_preventive_services_for_older_women', 'coronary_heart_disease',
       'current_asthma', 'current_smoking', 'dental_visit', 'diabetes',
       'health_insurance', 'high_blood_pressure', 'high_cholesterol',
       'mammography', 'mental_health', 'obesity', 'physical_health',
       'physical_inactivity', 'sleep_lt_7_hours', 'stroke',
       'taking_bp_medication', 'teeth_loss', 'ma_penetration_2020',
       'ma_penetration_2021', 'delta_ma_enrolled',
       'de

### Journaya Data

In [ ]:
J_sql = f"""
WITH jornaya_trunc AS (
SELECT *
FROM tracking.jornaya_event
),
boberdoo_publisher AS (
    SELECT leadid AS lead_id,
           product,
           lead_created,
           age,
           amount,
           state,
           source,
           lead_type,
           tcpa_universal_id
    FROM boberdoo.boberdoo
    WHERE product = 'MEDICARE'
    AND leadid IN {t_lead_ids}
) SELECT bp.lead_id AS boberdoo_lead_id,
         bp.product AS boberdoo_product,
         bp.age AS boberdoo_age,
         bp.state,
         bp.amount AS boberdoo_amount,
         bp.source AS boberdoo_source,
         bp.lead_type AS boberdoo_lead_type,
         bp.tcpa_universal_id AS boberdoo_tcpa_universal_id,
         tje.*
FROM boberdoo_publisher bp
LEFT JOIN jornaya_trunc tje
    ON bp.tcpa_universal_id = tje.tcpa_universal_id;
"""